# Predict Interaction Point in Unknown Input

In [ ]:
PATH_PREFIX = '../'
MODEL_SAVE_DIR = 'Model(Interaction Point)/'
MODEL_FILE = 'model.ckpt'
DATA_DIR = 'D0toPiKInAcceptanceSignal_Iteration1_Parsed/'
DATA_FILE_PATH = PATH_PREFIX + DATA_DIR + 'dataset_{:04d}.json'
OUTPUT_DIR = PATH_PREFIX + 'Prediction4/'
OUT_FILE_PATH = OUTPUT_DIR + 'result.csv'

SAVE_PREDICTION_IMAGES = False

FILE_NUMBER_TO_START_TESTING_FROM = 1
FILES_TO_TEST = 8456

NUMBER_OF_THREADS = 40

In [ ]:
#@title Imports

import sys
sys.path.append(PATH_PREFIX)

from __future__ import absolute_import, division, print_function

from graphGenerator import to_graph_dict_with_edges

import threading
import os.path
import ujson
import time

from graph_nets import utils_np, utils_tf
import modelTrack as model

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#@title Helper functions


def PlotModelPrediction(data,
                        true_positive,
                        true_negative,
                        false_positive,
                        title='Predicted Track by the Model',
                        scale=1,
                        show_layer=False,
                        save_loc=None):

    plt.close('all')
    fig = plt.figure(figsize=(4 * scale, 4 * scale), dpi=40 * scale)
    all_layers = [{'xx': [], 'yy': []} for _ in range(6)]
    for hits in data['tracks'].values():
        layers = [{'count': 0, 'x': 0, 'y': 0} for _ in range(6)]
        approx_track = {}
        for (layer, x, y) in hits:
            layers[layer]['count'] += 1
            layers[layer]['x'] += x
            layers[layer]['y'] += y
        for layer, pos in enumerate(layers):
            if pos['count']:
                approx_track[layer] = (1.0 * pos['x'] / pos['count'],
                                       1.0 * pos['y'] / pos['count'])
        for (layer, (x, y)) in approx_track.items():
            all_layers[layer]['xx'].append(x * scale)
            all_layers[layer]['yy'].append(y * scale)
            if show_layer:
                plt.annotate((layer + 1), (x * scale, y * scale))
    for layer, hits in enumerate(all_layers):
        plt.scatter(hits['xx'],
                    hits['yy'],
                    color='deepskyblue',
                    s=30,
                    alpha=1.0 * (layer + 1) / 6.0)
    for u, v in true_positive:
        plt.plot((u[0] * scale, v[0] * scale), (u[1] * scale, v[1] * scale),
                 linewidth=0.6,
                 color='green')
    for u, v in true_negative:
        plt.plot((u[0] * scale, v[0] * scale), (u[1] * scale, v[1] * scale),
                 ':',
                 linewidth=0.6,
                 color='green')
    for u, v in false_positive:
        plt.plot((u[0] * scale, v[0] * scale), (u[1] * scale, v[1] * scale),
                 linewidth=0.6,
                 color='red')
    plt.xlim(0, 1024 * scale)
    plt.ylim(0, 1024 * scale)
    plt.title(title)
    if save_loc:
        plt.savefig(save_loc)
        plt.close(fig)


def generate_dict_graphs(raw_data, offset, batch_size):
    input_graphs = []
    for i in range(offset, offset + batch_size):
        input_graph_dict, _, _, _, _ = to_graph_dict_with_edges(raw_data[i])
        input_graphs.append(input_graph_dict)
    return input_graphs


def create_placeholders(raw_data, offset, batch_size):
    input_graphs = generate_dict_graphs(raw_data, offset, batch_size)
    input_ph = utils_tf.placeholders_from_data_dicts(input_graphs)
    return input_ph


def create_feed_dict(raw_data, offset, batch_size, input_ph):
    inputs = generate_dict_graphs(raw_data, offset, batch_size)
    input_graphs = utils_np.data_dicts_to_graphs_tuple(inputs)
    feed_dict = {input_ph: input_graphs}
    return feed_dict


def classify_predicted_edges(data, prediction):
    true_positive, false_negative, false_positive = set([]), set([]), set([])
    for hits in data['tracks'].values():
        layers = [{'count': 0, 'x': 0, 'y': 0} for _ in range(6)]
        approx_track = {}
        for (layer, x, y) in hits:
            layers[layer]['count'] += 1
            layers[layer]['x'] += x
            layers[layer]['y'] += y
        for layer, pos in enumerate(layers):
            if pos['count']:
                approx_track[layer] = (1.0 * pos['x'] / pos['count'],
                                       1.0 * pos['y'] / pos['count'])
        prev_node, edges = None, {}
        for i, layer in enumerate(sorted(approx_track.keys())):
            edges[approx_track[layer]] = None
            if i != 0:
                edges[prev_node] = approx_track[layer]
            prev_node = approx_track[layer]
        for u, v in edges.items():
            if v is not None:
                if u in prediction or v in prediction:
                    if (u in prediction and v in prediction[u]) or (
                            v in prediction and u in prediction[v]):
                        true_positive.add((u, v))
                    else:
                        false_negative.add((u, v))
                    if u in prediction:
                        prediction[u].discard(v)
                    if v in prediction:
                        prediction[v].discard(u)
                else:
                    false_negative.add((u, v))
    for u in prediction:
        for v in prediction[u]:
            if (v, u) not in false_positive:
                false_positive.add((u, v))

    return true_positive, false_negative, false_positive

In [ ]:
#@title Restore Model

tf.reset_default_graph()

test_raw_data = ujson.loads(open(DATA_FILE_PATH.format(FILE_NUMBER_TO_START_TESTING_FROM), 'r').read())

input_ph = create_placeholders(test_raw_data, 0, 100)

num_processing_steps = 10

model = model.EncodeProcessDecode(edge_output_size=2)

test_outputs = model(input_ph, num_processing_steps)

input_ph = utils_tf.make_runnable_in_session(input_ph)

saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, PATH_PREFIX + MODEL_SAVE_DIR + MODEL_FILE)
print("Model restored")

In [ ]:
#@title Test Unknown Input

skipped_files = []
thread_pool = []
fpr, tpr = [], []
lock = threading.Lock()

if not os.path.isfile(OUT_FILE_PATH):
    out_file = open(OUT_FILE_PATH, 'w')
    out_file.write(
        'File number, Datapoint, # Track edges, # Track edges predicted by the model, # Track edges failed to predict, # Random edge incorrectly predicted as track edge, Accuracy\n'
    )
    out_file.close()
    
def testing_thread(fnum):
    f = open(DATA_FILE_PATH.format(FILE_NUMBER_TO_START_TESTING_FROM + fnum),
             'r')
    test_raw_data = ujson.loads(f.read())
    f.close()

    lock.acquire()
    print('testing file #{:04d} ...'.format(FILE_NUMBER_TO_START_TESTING_FROM +
                                            fnum),
          end=' ')
    start_time = time.time()
    try:
        feed_dict = create_feed_dict(test_raw_data, 0, 100, input_ph)
        test_values = sess.run({
            'inputs': input_ph,
            'outputs': test_outputs
        },
                               feed_dict=feed_dict)
    except Exception as e:
        print('skipped because of exception: {}'.format(e))
        skipped_files.append(FILE_NUMBER_TO_START_TESTING_FROM + fnum)
        lock.release()
        return
    else:
        elapsed_time = time.time() - start_time
        print('finished in {:.2f}s'.format(elapsed_time))
    lock.release()

    input = test_values['inputs']
    output = test_values['outputs'][-1]

    idds = utils_np.graphs_tuple_to_data_dicts(input)
    odds = utils_np.graphs_tuple_to_data_dicts(output)

    for i in range(100):
        dimension = 1024
        raw_data = test_raw_data[i]
        id = idds[i]
        od = odds[i]
        track_edges = np.argmax(od['edges'], axis=-1).astype(bool)
        predicted_track_edges = {}
        for j, edge in enumerate(id['edges']):
            if track_edges[j]:
                ux, uy, vx, vy = tuple((edge[1:5] * dimension).astype(float))
                if (ux, uy) not in predicted_track_edges:
                    predicted_track_edges[(ux, uy)] = set([])
                if (vx, vy) not in predicted_track_edges:
                    predicted_track_edges[(vx, vy)] = set([])
                predicted_track_edges[(ux, uy)].add((vx, vy))
                predicted_track_edges[(vx, vy)].add((ux, uy))

        true_positive, false_negative, false_positive = classify_predicted_edges(
            raw_data, predicted_track_edges)
        
        n_nodes = sum([len(track) for track in raw_data['tracks'].values()])
        n_edges = n_nodes * (n_nodes - 1) / 2
        n_true_negative = n_edges - len(true_positive) - len(false_positive) - len(false_negative)
        false_positive_rate = len(false_positive) * 1.0 / (len(false_positive) + n_true_negative)
        true_positive_rate = len(true_positive) * 1.0 / (len(true_positive) + len(false_negative))
        
        lock.acquire()
        fpr.append(false_positive_rate)
        tpr.append(true_positive_rate)
        lock.release()

        predicted_track_edges = true_positive.union(false_positive)

        # set of edges predicted to be track-edges by the model
        # print(predicted_track_edges)

        if len(false_negative) or len(false_positive):
            lock.acquire()
            out_file = open(OUT_FILE_PATH, 'a')
            out_file.write(
                '{:04d}, {:03d}, {:03d}, {:03d}, {:03d}, {:03d}, {:0.4f}\n'.
                format(
                    FILE_NUMBER_TO_START_TESTING_FROM + fnum, i + 1,
                    len(true_positive.union(false_negative)),
                    len(predicted_track_edges), len(false_negative),
                    len(false_positive),
                    len(true_positive) * 1.0 /
                    float(len(true_positive.union(false_positive)))))
            out_file.close()
            if SAVE_PREDICTION_IMAGES:
                PlotModelPrediction(
                    raw_data,
                    true_positive,
                    false_negative,
                    false_positive,
                    scale=4,
                    title=
                    '(Labelled) Prediction by the model for image {:04d}.{:03d}'
                    .format(FILE_NUMBER_TO_START_TESTING_FROM + fnum, i + 1),
                    save_loc=OUTPUT_DIR +
                    '{:04d}.{:03d}_prediction.png'.format(
                        FILE_NUMBER_TO_START_TESTING_FROM + fnum, i + 1))
            lock.release()

for fnum in range(FILES_TO_TEST):
    if len(thread_pool) == NUMBER_OF_THREADS:
        thread = thread_pool.pop(0)
        thread.join()
    new_thread = threading.Thread(target=testing_thread, args=(fnum, ))
    thread_pool.append(new_thread)
    new_thread.start()

for thread in thread_pool:
    thread.join()

f = open(OUTPUT_DIR + 'inconsistent.txt', 'w')
f.write('Inconsistent files skipped while testing: {}'.format(skipped_files))
f.close()
print('Output generated in: {}'.format(OUT_FILE_PATH))

plt.plot(fpr, tpr, 'ro', markersize=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

sess.close()